# Cassandra
environment setup

*should be able to remove mosdef and omnia channels sooooon*

`conda create -n cassandra -y -c conda-forge -c mosdef -c omnia “python=3.7” mbuild foyer openbabel py3dmol pytest`

`conda activate cassandra`

`python -m pip install ovito jupyterlab`

`git clone git@github.com:jennyfothergill/mosdef_cassandra.git`

`cd mosdef_cassandra`

`python -m pip install .`

In [ ]:
# Add Cassandra to your path
%%bash
export PATH=/Users/jenny/Projects/Cassandra_V1.2/bin:${PATH}
echo $PATH

In [1]:
import mosdef_cassandra as mc
import foyer
import mbuild as mb
import mosdef_cassandra.examples as ex

In [2]:
# If this fails, you need to add Cassandra to your path
mc.utils.detect_cassandra_binaries();

Using the following executables for Cassandra:
Python: /usr/bin/python2.7
library_setup: /Users/jenny/Projects/Cassandra_V1.2/bin/library_setup.py
Cassandra: /Users/jenny/Projects/Cassandra_V1.2/bin/cassandra_gfortran.exe


In [3]:
lattice = ex.structures.carbon_lattice()
#lattice.visualize(color_scheme={"_CH4": "grey"}).show()

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/lattice.py:584: UserWarning: Periodicity of non-rectangular lattices are not valid with default boxes. Only rectangular lattices are valid at this time.
  warn('Periodicity of non-rectangular lattices are not valid with '


In [4]:
methane = mb.load("C", smiles=True)
#methane.visualize().show()

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/compound.py:2749: UserWarning: No unitcell detected for pybel.Molecule C	

  warn("No unitcell detected for pybel.Molecule {}".format(pybel_mol))


In [5]:
# Load forcefields
trappe = foyer.forcefields.load_TRAPPE_UA()
oplsaa = foyer.forcefields.load_OPLSAA()

# Use foyer to apply forcefields
typed_lattice = trappe.apply(lattice)
typed_methane = oplsaa.apply(methane)

# Create box and species list
box_list = [lattice]
species_list = [typed_lattice, typed_methane]

# Since we have an occupied box we need to specify
# the number of each species present in the intial config
mols_in_boxes = [[1, 0]]

system = mc.System(box_list, species_list, mols_in_boxes=mols_in_boxes)
moves = mc.Moves("gcmc", species_list)

custom_args = {
    "chemical_potentials": ["none", -30.0],
    "rcut_min": 0.5,
    "vdw_cutoff": 14.0,
    "charge_cutoff": 14.0,
    "coord_freq": 100,
    "prop_freq": 10,
}

mc.run(system, moves, 300.0, "equilibration", 10000, **custom_args)

/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom element for _CH4
  'Creating custom element for {}'.format(element))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom element for _CH3
  'Creating custom element for {}'.format(element))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom element for _CH2
  'Creating custom element for {}'.format(element))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/forcefield.py:396: UserWarning: Non-atomistic element type detected. Creating custom element for _HC
  'Creating custom element for {}'.format(element))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/foyer/validator.py:132: Va

Treating <Structure 320 atoms; 1 residues; 0 bonds; PBC (orthogonal); NOT parametrized> as a non-insertable rigid species since it has no bonds
Using the following executables for Cassandra:
Python: /usr/bin/python2.7
library_setup: /Users/jenny/Projects/Cassandra_V1.2/bin/library_setup.py
Cassandra: /Users/jenny/Projects/Cassandra_V1.2/bin/cassandra_gfortran.exe
Generating fragment libraries...


/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:178: UserWarning: No bonds found. Cassandra will interpet this as a rigid species
  warnings.warn("No bonds found. Cassandra will interpet "
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:564: UserWarning: More than two atoms present but no fragments identified.
  warnings.warn('More than two atoms present but '
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:303: UserWarning: Warning, type name opls_138 will be shortened to six characters as ls_138. Please confirm your final MCF.
  "MCF.".format(itype,itype[-6:]))
/Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/mbuild/formats/cassandramcf.py:303: UserWarning: Warning, type name opls_140 will be shortened to six characters as ls_140. Please confirm your final MCF.
  "MCF.".format(itype,itype[-6:]))
/Users/jenny/miniconda3

Running Cassandra...


In [12]:
%%file ovito_render.py

def ovito_render():
    "Render images using OVITO"
    import ovito
    from ovito.io import import_file
    from ovito.vis import Viewport, RenderSettings, TachyonRenderer
    from ovito.modifiers import ColorCodingModifier
    from ovito.modifiers import PolyhedralTemplateMatchingModifier
    
    pipeline = import_file("gcmc.out.xyz")
    #ptm_modifier = PolyhedralTemplateMatchingModifier(
    #    color_by_type=True,
    #    rmsd_cutoff=0.3,
    #)
    #for structure_type in ['BCC', 'CUBIC_DIAMOND', 'FCC', 'GRAPHENE',
    #                       'HCP', 'HEX_DIAMOND', 'ICO', 'OTHER', 'SC']:
    #    structure_type = getattr(PolyhedralTemplateMatchingModifier.Type, structure_type)
    #    ptm_modifier.structures[structure_type].enabled = True
    #pipeline.modifiers.append(ptm_modifier)
    data_collection = pipeline.compute()
    pipeline.add_to_scene()
    renderer = TachyonRenderer(antialiasing_samples=32,)
    vp = Viewport(type=Viewport.Type.Perspective, camera_dir=(2, 1, -1))
    vp.zoom_all()
    vp.camera_pos = np.asarray(vp.camera_pos) * np.array([0.8, 0.8, 0.75])
    vp.render_image(
        filename='sample.png',
        size=(1000, 750),
        frame=pipeline.source.num_frames-1,
        alpha=True,
        renderer=renderer,
    )
    every_nth = pipeline.source.num_frames // 20
    video = vp.render_anim(
        filename='sample.gif',
        size=(480, 320),
        fps=5,
        renderer=renderer,
        every_nth=every_nth,
    )
    image = vp.render_image(size=(800, 600), filename='img.png', frame=pipeline.source.num_frames)
    pipeline.remove_from_scene()
    

if __name__ == "__main__":
    ovito_render()

Overwriting ovito_render.py


In [13]:
!python ovito_render.py

objc[4018]: Class QMacAutoReleasePoolTracker is implemented in both /Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/PySide2/Qt/lib/QtCore.framework/Versions/5/QtCore (0x111f43030) and /Users/jenny/miniconda3/envs/cassandra/lib/libQt5Core.5.12.5.dylib (0x11bced560). One of the two will be used. Which one is undefined.
objc[4018]: Class QT_ROOT_LEVEL_POOL__THESE_OBJECTS_WILL_BE_RELEASED_WHEN_QAPP_GOES_OUT_OF_SCOPE is implemented in both /Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/PySide2/Qt/lib/QtCore.framework/Versions/5/QtCore (0x111f430a8) and /Users/jenny/miniconda3/envs/cassandra/lib/libQt5Core.5.12.5.dylib (0x11bced5d8). One of the two will be used. Which one is undefined.
objc[4018]: Class KeyValueObserver is implemented in both /Users/jenny/miniconda3/envs/cassandra/lib/python3.7/site-packages/PySide2/Qt/lib/QtCore.framework/Versions/5/QtCore (0x111f430d0) and /Users/jenny/miniconda3/envs/cassandra/lib/libQt5Core.5.12.5.dylib (0x11bced600

In [11]:
!ls

Adsorption_example.ipynb
README.md
box1.in.xyz
ex_render.py
gcmc.inp
gcmc.out.H
gcmc.out.chk
gcmc.out.log
gcmc.out.prp
gcmc.out.xyz
mosdef_cassandra_2020-03-03_14:19:14.764804.log
ovito_render.py
species1
species1.mcf
species1.pdb
species2
species2.mcf
species2.pdb


In [10]:
# mc.print_valid_kwargs()

In [8]:
# gcmc.out.H - box
# gcmc.out.xyz - particle coordinates
# species#.mcf - connectivity
# Cassandra treats no bonds as fixed - check the inp file (max distances for the frozen species should be 0)
# this file species2/frag1/frag1.dat contains the fragment configurations 
# --changing the temp (300) to natoms (5 for methane) makes it viewable by vmd (watch the fragment wiggle around)
# chk - restart (mc.restart)
# log - energies, etc
# properties - thermo properties
# cassandra uses fixed bonds -- make sure ring systems have correct bonds!
!ls

Adsorption_example.ipynb
README.md
box1.in.xyz
ex_render.py
gcmc.inp
gcmc.out.H
gcmc.out.chk
gcmc.out.log
gcmc.out.prp
gcmc.out.xyz
mosdef_cassandra_2020-03-03_14:19:14.764804.log
species1
species1.mcf
species1.pdb
species2
species2.mcf
species2.pdb


In [7]:
#%%bash
#for file in gcmc.out*
#do
#echo "!!! FILE" $file;
#head -50 $file
#done

In [16]:
!head species2/frag1/frag1.dat

      100000
   300.00000000000000        308.96094708967013     
 ls   0.0000000000000000        0.0000000000000000        0.0000000000000000     
 ls -0.28611858535217743      -0.98085726486241431      -0.37967773319318276     
 ls  0.54601513042094063      -0.15914456848617042       0.92986042903450894     
 ls  0.65381729971336111       0.47762771508257179      -0.72972235591775014     
 ls -0.82688975448963709       0.66223193612725539       0.25651944183026865     
   300.00000000000000        369.88588330996089     
 ls   0.0000000000000000        0.0000000000000000        0.0000000000000000     
 ls -0.42191781948382390      -0.90885453579680331      -0.42903238381690284     
